In [210]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [261]:
df_wykop = pd.read_csv('BAN-PL.csv')
df_wykop

,id,Text,Class
0,2200,\n\n\n Polska wtedy oficjalnie powi...,0
1,2201,\n Gigantyczna różnica\n\n{USERNAME}: biorac ...,0
2,2202,\n\n {USERNAME}: Moj kumpel budowla...,0
3,2203,"kura, rzodkiewka za 3pln to nie jest tanio",0
4,2204,{USERNAME}: większość nie idzie w marszu za PO...,0
...,...,...,...
23995,12563,Polaczki jebane XD na wszystko narzekać byle n...,1
23996,12564,jakaś muzyka normicka dudni ehhh w sumie i tak...,1
23997,12565,"{USERNAME}: ""[pseudonym]"" czy raczej zjeb ze s...",1
23998,NaN,{USERNAME} {USERNAME} kurwy Michał [surname] k...,1


In [262]:
from datasets import load_dataset

dataset = load_dataset("Paul/hatecheck-polish")

In [268]:
df_hatecheck = pd.DataFrame(dataset['test'])
df_hatecheck['Text'] = df_hatecheck['test_case']
df_hatecheck['Class'] = df_hatecheck['label_gold'].map({'hateful': 1, 'non-hateful': 0})

df_hatecheck = df_hatecheck[['Text', 'Class']]
df_hatecheck

,Text,Class
0,Nienawidzę gejów.,1
1,Nienawidzę azjatów.,1
2,Nienawidzę niepełnosprawnych.,1
3,Nienawidzę Żydów.,1
4,Nienawidzę imigrantów.,1
...,...,...
3810,Geje są wszyscy kompletnymi dupk4mi.,1
3811,Azjaci są wszyscy kompletnymi dupk4mi.,1
3812,Niepełnosprawni są wszyscy kompletnymi dupk4mi.,1
3813,Żydzi są wszyscy kompletnymi dupk4mi.,1


In [269]:
df = pd.concat([df_wykop[['Text', 'Class']], df_hatecheck[['Text', 'Class']]])

df

,Text,Class
0,\n\n\n Polska wtedy oficjalnie powi...,0
1,\n Gigantyczna różnica\n\n{USERNAME}: biorac ...,0
2,\n\n {USERNAME}: Moj kumpel budowla...,0
3,"kura, rzodkiewka za 3pln to nie jest tanio",0
4,{USERNAME}: większość nie idzie w marszu za PO...,0
...,...,...
3810,Geje są wszyscy kompletnymi dupk4mi.,1
3811,Azjaci są wszyscy kompletnymi dupk4mi.,1
3812,Niepełnosprawni są wszyscy kompletnymi dupk4mi.,1
3813,Żydzi są wszyscy kompletnymi dupk4mi.,1


In [270]:
text_data = df['Text']
target = df['Class']

target.value_counts()

Class
1    14678
0    13137
Name: count, dtype: int64

In [271]:
X_train, X_test, y_train, y_test = train_test_split(text_data, target, test_size=0.2, random_state=42)

In [272]:
vectorizer = TfidfVectorizer(max_features=5000) 
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [273]:
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [274]:
predictions = model.predict(X_test_tfidf)

In [275]:
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

In [276]:
print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

Accuracy: 0.8324644975732518
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.84      0.83      2616
           1       0.86      0.82      0.84      2947

    accuracy                           0.83      5563
   macro avg       0.83      0.83      0.83      5563
weighted avg       0.83      0.83      0.83      5563



In [277]:
import joblib

joblib.dump(model, 'model.joblib')

['model.joblib']

In [278]:
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')

['tfidf_vectorizer.joblib']